# Deli in vladaj

1. nalogo **razdelimo** na **za faktor manjše** podnaloge
2. podnaloge **rekurzivno rešimo**
3. dobljene rešitve **združimo** v rešitev prvotne naloge (lastnost optimalne podstrukture)

![](../../zapiski/slike/deli-in-vladaj.png)

## Hitro potenciranje

In [ ]:
let rec potenciraj a =
  function
  | 0 -> 1
  | n -> a * potenciraj a (n - 1)

In [2]:
let kvadriraj x = x * x

val kvadriraj : int -> int = <fun>


In [3]:
let rec potenciraj' a =
  function
  | 0 -> 1
  | n ->
      if n mod 2 = 0 then
        kvadriraj @@ potenciraj' a (n / 2)
      else
        a * (kvadriraj @@ potenciraj' a ((n - 1) / 2))

val potenciraj' : int -> int -> int = <fun>


In [5]:
potenciraj' 7 42

- : int = -61959248125573647


### Fibonaccijeva števila

In [ ]:
let ostanek = 10000000000

let rec fib = function
  | 0 -> 0
  | 1 -> 1
  | n -> fib (n - 1) + fib (n - 2)

let fib' =
  let rec aux a b = function
  | 0 -> a
  | n -> aux (b mod ostanek) ((a + b) mod ostanek) (n - 1)
  in
  aux 0 1

In [ ]:
fib 10

In [ ]:
fib' 10

In [ ]:
type matrika = {
    _11: int; _12: int; _21: int; _22: int
}

let zmnozi a b = {
    _11 = a._11 * b._11 + a._12 * b._21;
    _21 = a._21 * b._11 + a._22 * b._21;
    _12 = a._11 * b._12 + a._12 * b._22;
    _22 = a._21 * b._12 + a._22 * b._22
}

let ostanek_matrike a m = {
    _11 = a._11 mod m;
    _12 = a._12 mod m;
    _21 = a._21 mod m;
    _22 = a._22 mod m
}

## Urejanje

In [6]:
let seznam_nakljucnih m n = List.init n (fun _ -> Random.int m)

val seznam_nakljucnih : int -> int -> int list = <fun>


In [7]:
let stopaj f x =
  let zacetek = Sys.time () in
  let _ = f x in
  let konec = Sys.time () in
  let izpis =
    Printf.sprintf "Porabljen čas: %f ms" (1000. *. (konec -. zacetek))
  in
  print_endline izpis

val stopaj : ('a -> 'b) -> 'a -> unit = <fun>


In [8]:
let rec vstavi y = function
  | [] -> [ y ]
  | x :: xs when y > x -> x :: vstavi y xs
  | x :: xs -> y :: x :: xs

let uredi_z_vstavljanjem sez =
  List.fold_left (fun ze_urejen x -> vstavi x ze_urejen) [] sez

val vstavi : 'a -> 'a list -> 'a list = <fun>


val uredi_z_vstavljanjem : 'a list -> 'a list = <fun>


In [9]:
let rezultati =
  [ 1000; 2000; 4000; 8000; 16000 ]
  |> List.map (fun n -> seznam_nakljucnih n n)
  |> List.iter (stopaj uredi_z_vstavljanjem)

Porabljen čas: 7.595000 ms
Porabljen čas: 29.059000 ms
Porabljen čas: 74.676000 ms
Porabljen čas: 288.386000 ms
Porabljen čas: 1366.707000 ms


val rezultati : unit = ()


## Urejanje z zlivanjem

In [11]:
let rec zlij xs ys =
  match xs, ys with
  | [], _ -> ys
  | _, [] -> xs
  | x :: xs', y :: ys' ->
      if x <= y then
        x :: zlij xs' ys
      else
        y :: zlij xs ys'

val zlij : 'a list -> 'a list -> 'a list = <fun>


In [12]:
zlij [1; 2; 5; 7] [1; 3; 4; 6; 6]

- : int list = [1; 1; 2; 3; 4; 5; 6; 6; 7]


In [13]:
zlij [5; 1; 7; 2] [6; 4; 6; 1; 3]

- : int list = [5; 1; 6; 4; 6; 1; 3; 7; 2]


In [19]:
let razpolovi xs =
  let xs1, xs2 =
    xs
    |> List.mapi (fun i x -> (i, x))
    |>
      let n = List.length xs in 
      List.partition (fun (i, _) -> i < n / 2)
  in
  (List.map snd xs1, List.map snd xs2)

val razpolovi : 'a list -> 'a list * 'a list = <fun>


In [20]:
razpolovi [5; 1; 7; 2; 6; 4; 6; 1; 3]

- : int list * int list = ([5; 1; 7; 2], [6; 4; 6; 1; 3])


In [25]:
let rec uredi_z_zlivanjem =
  function
  | [] -> []
  | [x] -> [x]
  | xs ->
      let xs1, xs2 = razpolovi xs in
      zlij (uredi_z_zlivanjem xs1) (uredi_z_zlivanjem xs2)

val uredi_z_zlivanjem : 'a list -> 'a list = <fun>


In [26]:
uredi_z_zlivanjem [5; 1; 7; 2; 6; 4; 6; 1; 3]

- : int list = [1; 1; 2; 3; 4; 5; 6; 6; 7]


In [30]:
let rezultati =
  [ 1000; 2000; 4000; 8000; 16000; 32000; 64000 ]
  |> List.map (fun n -> seznam_nakljucnih n n)
  |> List.iter (stopaj uredi_z_zlivanjem)

Porabljen čas: 4.633000 ms
Porabljen čas: 4.434000 ms
Porabljen čas: 9.916000 ms
Porabljen čas: 21.526000 ms
Porabljen čas: 33.970000 ms
Porabljen čas: 74.736000 ms
Porabljen čas: 155.519000 ms


val rezultati : unit = ()


## Hitro urejanje

In [31]:
let rec hitro_uredi =
  function
  | [] -> []
  | x :: xs ->
      let manjsi, vecji = List.partition (fun y -> y <= x) xs in
      hitro_uredi manjsi @ (x :: hitro_uredi vecji)

val hitro_uredi : 'a list -> 'a list = <fun>


In [33]:
let rezultati =
  [ 1000; 2000; 4000; 8000; 16000 ]
  |> List.map (fun n -> List.init n (fun i -> i))
  |> List.iter (stopaj hitro_uredi)

Porabljen čas: 38.858000 ms
Porabljen čas: 122.970000 ms
Porabljen čas: 513.519000 ms
Porabljen čas: 2287.881000 ms
Porabljen čas: 9935.443000 ms


val rezultati : unit = ()


## Hitro urejanje na mestu

In [ ]:
let tabela_nakljucnih m n = Array.init n (fun _ -> Random.int m)

In [ ]:
let zamenjaj tab i j =
  let t = tab.(i) in
  tab.(i) <- tab.(j);
  tab.(j) <- t

In [ ]:
let pivotiraj_manjsi_neurejeni_vecji tabela zacetek konec =
  let pivot = tabela.(zacetek) and l = ref (zacetek + 1) and d = ref konec in
  while !l < !d do
    while !l < !d && tabela.(!l) <= pivot do
      incr l
    done;
    while !l < !d && tabela.(!d) > pivot do
      decr d
    done;
    zamenjaj tabela !l !d
  done;
  let p = if tabela.(!l) <= pivot then !l else !l - 1 in
  zamenjaj tabela zacetek p;
  p

In [ ]:
let pivotiraj_manjsi_vecji_neurejeni tabela zacetek konec =
  let pivot = tabela.(zacetek) and zacetek_vecjih = ref (zacetek + 1) in
  for i = zacetek + 1 to konec do
    if tabela.(i) <= pivot then (
      zamenjaj tabela i !zacetek_vecjih;
      incr zacetek_vecjih)
  done;
  let p = !zacetek_vecjih - 1 in
  zamenjaj tabela zacetek p;
  p

## Urejanje s primerjavo je $\Omega(n \log n)$

![](../../zapiski/slike/odlocitveno-drevo.png)